In [1]:
import cv2
import mediapipe as mp
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt


In [2]:
# 🎥 Capture a photo from webcam
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
if ret:
    cv2.imwrite('face.jpg', frame)
cap.release()
cv2.destroyAllWindows()


In [4]:
# 🔍 Detect facial landmarks
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)
image = cv2.imread('face.jpg')
rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
results = face_mesh.process(rgb)

h, w, _ = image.shape
landmarks = []
if results.multi_face_landmarks:
    for lm in results.multi_face_landmarks[0].landmark:
        x, y = int(lm.x * w), int(lm.y * h)
        landmarks.append((x, y))

# Draw landmarks
for (x, y) in landmarks:
    cv2.circle(image, (x, y), 1, (0, 255, 0), -1)

cv2.imshow('Landmarks', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [7]:
# 🤖 Dummy classifier for face shape based on landmark ratios
# Replace these values with real calculations from the landmarks
example_features = [0.8, 1.0, 1.2]  # [jaw_ratio, cheek_ratio, length_ratio]
# Feature vector format: [jaw_ratio, cheekbone_ratio, face_length_ratio]
X_train = [
    [0.8, 1.0, 1.2],  # Round
    [0.9, 1.0, 1.5],  # Oval
    [1.0, 1.1, 1.1],  # Square
    [0.7, 1.0, 0.9],  # Heart
    [0.6, 1.2, 1.4]   # Diamond ← NEW
]

y_train = [
    'Round',
    'Oval',
    'Square',
    'Heart',
    'Diamond'  # ← NEW
]


knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)
predicted_shape = knn.predict([example_features])[0]

print("Detected Face Shape:", predicted_shape)


Detected Face Shape: Round


In [9]:
# 💇 Suggest hairstyles based on face shape
haircut_db = {
    'Round': [
        'Textured Crop',
        'Messy Fringe',
        'Angular Fringe',
        'Undercut with Volume',
        'Faux Hawk'
    ],
    'Oval': [
        'Buzz Cut',
        'Curtain Hair',
        'Quiff',
        'Taper Fade with Design',
        'Bro Flow'
    ],
    'Square': [
        'Pompadour',
        'Side Part with Fade',
        'French Crop',
        'Ivy League Cut',
        'Undercut with Sharp Edges'
    ],
    'Heart': [
        'Messy Quiff',
        'Low Fade with Fringe',
        'Tapered Brush Up',
        'Disconnected Undercut',
        'Side Swept Bangs'
    ],
    'Diamond': [
        'Layered Fringe',
        'Brush Up with Mid Fade',
        'Wavy Crop',
        'Faux Hawk Fade',
        'Slick Back'
    ]
}

# Get suggestions for the predicted shape
suggestions = haircut_db.get(predicted_shape, ['Classic Cut'])

# Print them
print("Recommended Hairstyles:")
for style in suggestions:
    print('-', style)


Recommended Hairstyles:
- Textured Crop
- Messy Fringe
- Angular Fringe
- Undercut with Volume
- Faux Hawk
